In [1]:
from _build import dataset_from_config
from dptb.utils.config import Config

In [2]:
config = {
    "root": "/root/nequip_data/",
    "dataset": "npz",
    "dataset_file_name": "/root/nequip_data/Si8-100K.npz",
    # "include_keys":[
    #     "kpoints",
    #     "eigenvalues"
    # ],
    "key_mapping":{
        "pos":"pos",
        "atomic_numbers":"atomic_numbers",
        "kpoints": "kpoint",
        "pbc": "pbc",
        "cell": "cell",
        "eigenvalues": "eigenvalue"
    },
    "npz_fixed_field_keys": ["kpoint", "pbc"],
    "graph_field":["eigenvalues"],
    "chemical_symbols": ["Si", "C"],
    "r_max": 4.0,
    "er_max": 3.0
}

config = Config(config=config)
# dataset: npz                                                                       # type of data set, can be npz or ase
# dataset_url: http://quantum-machine.org/gdml/data/npz/toluene_ccsd_t.zip           # url to download the npz. optional
# dataset_file_name: ./benchmark_data/toluene_ccsd_t-train.npz                       # path to data set file
# key_mapping:
#   z: atomic_numbers                                                                # atomic species, integers
#   E: total_energy                                                                  # total potential eneriges to train to
#   F: forces                                                                        # atomic forces to train to
#   R: pos                                                                           # raw atomic positions
# npz_fixed_field_keys:                                                              # fields that are repeated across different examples
#   - atomic_numbers

# chemical_symbols:
#   - H
#   - C

In [3]:
dataset = dataset_from_config(config=config, prefix="dataset")

In [8]:
from dataloader import DataLoader
for i,x in enumerate(dataset):
    dataset[i]["edge_vectors"] = x.get_edge_vectors()
print(dataset[1]["edge_vectors"])
loader = DataLoader(dataset, batch_size=3, shuffle=False)

None


In [7]:
import torch
from dptb.data.AtomicDataDict import with_edge_vectors
for data in loader:
    index = torch.arange(0,data.num_edges)[
        (data.batch[data.edge_index[0]]==0) + (data.batch[data.edge_index[1]]==0)
        ]
    
    print(data.eigenvalue.shape)
    break

torch.Size([3, 354, 60])


In [10]:
from dptb.data.AtomicDataDict import with_edge_vectors
for data in loader:
    print(data["edge_index"])
    break

tensor([[ 0,  0,  0,  ..., 23, 23, 23],
        [ 1,  6,  2,  ..., 20, 21, 22]])


In [1]:
import numpy as np

npzdata = np.load("/root/nequip_data/toluene_ccsd_t-test.npz")

In [2]:
print(npzdata.files)

deeptb_data = {}

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']


In [19]:
print(npzdata["type"])

b'd'


In [20]:
from ase import Atoms
from ase.io.trajectory import Trajectory
import numpy as np


In [21]:
Traj = Trajectory("/data/band/IV/MD/100K/SiC/kpath.0/xdat.traj")
eigenvalues = np.load("/data/band/IV/MD/100K/SiC/kpath.0/eigs.npy")
kpoints = np.load("/data/band/IV/MD/100K/SiC/kpath.0/kpoints.npy")

In [22]:
import torch
cell = []
pos = []
atomic_numbers = []
for i in Traj:
    cell.append(i.cell.array)
    pos.append(i.positions)
    atomic_numbers.append(i.get_atomic_numbers())
cell = np.array(cell)
pos = np.array(pos)
atomic_numbers = np.array(atomic_numbers)
    

In [23]:
name = "Si8-100K"
np.savez("/root/nequip_data/Si8-100K.npz", cell=cell, pos=pos, atomic_numbers=atomic_numbers, kpoints=kpoints, eigenvalues=eigenvalues, pbc=[True, True, True])

In [1]:
import torch

In [27]:
import time

start = time.time()
na = torch.nested.nested_tensor([torch.randn(3,3) for n in range(12000)])
mid = time.time()
torch.bmm(na, na)

end = time.time()
print(mid-start, end-start)

0.1382765769958496 0.1777637004852295


In [ ]:
out = []
start = time.time()
for n in range(12000):
    m = torch.randn(3,3)
    out.append(m @ m)

end = time.time()
print(end-start)

0.1254711151123047


In [13]:
mp = torch.bmm(na, na)
print(mp[0] - a @ a)
print(mp[1] - b @ b)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [17]:
print(mp.is_nested, a.is_nested)

True False
